<h1 align="center">word2vec 使用入門（運用維基百科文本作為訓練語料）</h1>
<hr>
<h3>1. 取得維基百科語料（目前訓練使用 2021-06-10 版本，建議用最新版）</h3>
<p><a href="https://dumps.wikimedia.org/zhwiki/">https://dumps.wikimedia.org/zhwiki/</a></p>
<p>
要下載的是以 <strong style="color:red">pages-articles.xml.bz2</strong> 結尾的備份，
而不是以 pages-articles-multistream.xml.bz2 結尾的備份，
否則會在清理上出現一些異常，無法正常解析文章。
</p>
<p>壓縮檔案超過 2G，解壓所後超過 9G（2021-06-10）</p>
<hr>
<p>
這個專案的資料檔都非常大，避免 Dropbox 同步困難，若以 windows 作為工作平台，建議所有資料檔集中在本機，如：
<strong style="color:magenta">c:/python/wiki</strong> 資料夾下。
</p>

<hr>
<h3>2. 安裝 gensim 工具模組（含 word2vec）</h3>
<pre>
<span style="color:orange">conda install gensim</span>
或
<span style="color:orange">pip3 install gensim</span>
</pre>
<pre>
注意：
<span style="color:red">建議重新建立新的虛擬環境，安裝最新版本 gensim（目前 conda 最新是 4.0.1 版）</span>，
若是舊環境增加模組安裝時，需注意版本問題，
gensim 更新至 3.8.0 版以上測試正常（之前的舊版測試時發現有錯誤訊息），
連帶 smart_open 與 numpy 也都需要更新，
smart_open 更新為 1.8.4 版，
numpy 更新為 1.17.2 版
</pre>

In [2]:
import gensim
print(gensim.__version__)

4.0.1


<hr>
<h3>3. 將 XML 去除標籤輸出成 TXT 純文字檔（耗時約一個小時）</h3>
<pre>
讀取 wiki XML 文件檔（目前是採用 <strong style="color:red">zhwiki-20210601-pages-articles.xml.bz2</strong>），
刪除 XML 標籤與英文文數字與符號（非中文）的所有文字，
輸出成純文字檔 <strong style="color:red">wiki_texts.txt</strong>（約 1.2G）</pre>

In [ ]:
# 將 XML 去除標籤輸出成 TXT 純文字檔

import gensim
import re

# 定義函數：去除非中文的文字與符號（只保留中文字）

def remove_punctuation(line):
    rule = re.compile(r'[^\u4e00-\u9fa5|\s]')
    line = rule.sub('', line)
    return line

# 定義函數：去除多餘的空白符號（只留下一個空白當作間隔）

def remove_redundant_space(line):
    line = re.sub(' +', ' ', line)
    return line

# 下載的 wiki 語料檔
# wiki_file = 'c:/python/wiki/zhwiki-20190901-pages-articles.xml.bz2'
wiki_file = 'c:/python/wiki/zhwiki-20210601-pages-articles.xml.bz2'

with open('c:/python/wiki/wiki_texts.txt', 'w', encoding='utf8') as fp:
    # 利用 gensim 載入
    # wiki = gensim.corpora.WikiCorpus(wiki_file, lemmatize=False, dictionary={}) # 舊版，新版不支援 lemmatize
    wiki = gensim.corpora.WikiCorpus(wiki_file, dictionary={})
    # 取出文字部分（原本是 XML 格式，包含很多標籤）
    for text in wiki.get_texts():
        # print(text)
        # text 是一篇文章，表示成字串串列（List）
        # text 中的字串連接合併成長字串，以空白字元作為間隔
        s = ' '.join(text)
        # 僅保留中文
        t = remove_punctuation(s)
        # 只留下一個空白當作間隔
        u = remove_redundant_space(t)
        # 每篇文章一個換行作為間隔，寫入輸出檔案
        fp.write(u + '\n')

fp.close()


<hr>
<h3>4. 利用 OpenCC 將 TXT 純文字內容都轉換為繁體（必要時，耗時超過一個小時）</h3>
<pre>
在 Linux 平台安裝 opencc：
<span style="color:orange">apt install opencc</span>
執行：
<span style="color:orange">opencc -i wiki_texts.txt -o wiki_zh_tw.txt -c s2tw.json</span>
（在 Linux 處理速度快很多）
最後，輸出成繁體的 <strong style="color:red">wiki_zh_tw.txt</strong> 文字檔
</pre>

<hr>
<h3>也可以安裝 python 的 opencc 工具模組來進行繁簡轉換</h3>
<pre>
opencc 是繁簡轉換工具模組
安裝：（目前 conda 不提供此模組）
<span style="color:orange">pip3 install opencc-python-reimplemented</span>
</pre>

In [ ]:
# 運用 python 的 opencc 工具模組執行繁體轉換

from opencc import OpenCC

openCC = OpenCC('s2t')

with open('c:/python/wiki/wiki_texts.txt', 'r', encoding='utf-8') as fp:
    s = fp.read()
fp.close()

t = openCC.convert(s)

with open('c:/python/wiki/wiki_zh_tw.txt', 'w', encoding='utf-8') as fp:
    fp.write(t)
fp.close()

print('Conversion Complete!')


<hr>
<h3>5. 利用 jieba 工具模組進行斷詞（非常耗時的計算）</h3>
<pre>
安裝：
<span style="color:orange">conda install -c conda-forge jieba</span>
或
<span style="color:orange">pip3 install jieba</span>
最後，輸出成分詞（斷詞）後的 <strong style="color:red">wiki_seg.txt</strong> 文字檔
</pre>

In [ ]:
# 利用 jieba 工具模組進行斷詞
#
# 目前沒有設定自用辭典（user diectionary）
# jieba.load_userdict(file_name)
# 特定專有術語應該放入此辭典中，如：CIS 180 image words
# 目前也沒有設定 stopwords
# 若除去英文（非中文）字元後，至少還應該除去全形標點符號
#

import jieba
import logging

def main():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    # 支援繁體文句分詞的專用辭典
    jieba.set_dictionary('dict.txt.big')
    # 載入額外的使用者辭典
    # jieba.load_userdict('ccom223_iwdb.txt')

    # 輸出分詞結果
    ofile = open('c:/python/wiki/wiki_seg.txt', 'w', encoding='utf-8')

    n = 0
    
    with open('c:/python/wiki/wiki_zh_tw.txt' ,'r', encoding='utf-8') as ifile:
        for line in ifile:
            words = jieba.cut(line, cut_all=False)
            for w in words:
                ofile.write(w + ' ')
                n = n + 1
                if (n % 1000000 == 0):
                    # print('已完成前 %d 行的斷詞' % n)
                    logging.info('已完成前 %d 行的斷詞' % n)

    ifile.close()
    ofile.close()

if __name__ == '__main__':
    main()


<hr>
<h3>6. word2vec 訓練（稍微耗時的計算）</h3>
<pre>
vector 長度：<strong style="color:red">250-400</strong>
最後，輸出 word2vec model 檔：<strong style="color:red">wiki.model.bin</strong>
</pre>
<pre>
<span style="color:orange">model = word2vec.Word2Vec(sentences, size=400, iter=10, min_count=1)</span>
size=400 # word vector 維度（視需要而定，一般中文語料 250-400）
iter=10 # 訓練回合數（epoch，預設 5）
min_count=1 # 出現詞頻數（超過這個次數的詞就會被列入辭典）
</pre>

In [ ]:
# 訓練 word2vec model
#

from gensim.models import word2vec
import logging

def main():
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    sentences = word2vec.Text8Corpus('c:/python/wiki/wiki_seg.txt')
    model = word2vec.Word2Vec(sentences, size=400)
    # Save our model.
    model.save('c:/python/wiki/wiki.model.bin')
    # To load a model.
    # model = word2vec.Word2Vec.load('c:/python/wiki/wiki.model.bin')

if __name__ == "__main__":
    main()


<hr>
<h3>7. 測試（相似詞測試、相似度測試、詞向量）</h3>

<hr>
<h3>載入 word2vec model</h3>

In [ ]:
# 載入 word2vec model

import gensim

print(gensim.__version__)

model = gensim.models.Word2Vec.load('c:/python/wiki/wiki.model.bin')


<hr>
<h3>相似詞測試</h3>

In [ ]:
# 相似詞測試

q = '快樂'

try:
    lst = model.wv.most_similar(q)
except:
    print('No %s in corpus!' % q)
    lst = []

for i in lst:
    t, w = i
    print('%20.16f: %s' % (w, t))


<hr>
<h3>相似度測試</h3>

In [ ]:
# 相似度測試

q1 = '快樂'
q2 = '高興'

try:
    s = model.wv.similarity(q1, q2)
except:
    s = 0
    print('無從測試!')

print(s)


<hr>
<h3>詞向量與餘弦距離計算</h3>

In [ ]:
# 詞向量與餘弦距離計算

import gensim
import math

q1 = '巴洛克'
q2 = '古典'

sm = model.wv.similarity(q1, q2)

# KeyedVectors Instance gets stored
# v1 = model.wv.word_vec(q1)
# v2 = model.wv.word_vec(q2)
v1 = model.wv.get_vector(q1)
v2 = model.wv.get_vector(q2)

print(q1, '=')
print(v1)
print(q2, '=')
print(v2)

# Cosine value of word vectors
s0, s1, s2 = 0.0, 0.0, 0.0
for i in range(len(v1)):
    s0 = s0 + (v1[i] * v2[i])
    s1 = s1 + (v1[i] * v1[i])
    s2 = s2 + (v2[i] * v2[i])
s1 = math.sqrt(s1)
s2 = math.sqrt(s2)
cs = s0 / (s1 * s2)

print('similarity =', sm)
print('    cosine =', cs)

# for i in range(len(v1)):
#     t = '%3d. %8.4f %8.4f' % (i, v1[i], v2[i])
#     print(t)


<hr>
<h3>以下為暫時性測試</h3>

In [ ]:
# OpenCC 測試

from opencc import OpenCC

# convert from Simplified Chinese to Traditional Chinese
openCC = OpenCC('s2t')

# can also set conversion by calling set_conversion
# openCC.set_conversion('s2tw')

chs= '开放中文转换'
cht= openCC.convert(chs)

print(cht)


In [ ]:
# 去除所有半角全角符号，只留字母、数字、中文。

import re

def remove_punctuation(line):
    rule = re.compile(r'[^\u4e00-\u9fa5|\s]')
    line = rule.sub('', line)
    return line

def remove_redundant_space(line):
    line = re.sub(' +', ' ', line)
    return line

s = '开放中文转换 abc, XyZ#$%， 塵土    123 飛揚'

t = remove_punctuation(s)
u = remove_redundant_space(t)

print(u)


In [ ]:
s = '中文字   空白   刪除 ！'
t = re.sub(' +', ' ', s)
print(t)